# Fake News Detection

## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

## Importing Dataset

In [3]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [4]:
df_1 = pd.read_csv('train.csv', sep = ";")
df_2 = pd.read_csv('train_2.csv', sep = ";")
df_3 = pd.read_csv('train_4.csv', sep = ";")
df_4 = pd.read_csv('train_5.csv', sep = ";")
df_5 = pd.read_csv('test.csv', sep = ";")
df_6 = pd.read_csv('test_2.csv', sep = ";")
df_7 = pd.read_csv('test_3.csv', sep = ";")
df_8 = pd.read_csv('test_5.csv', sep = ";")

In [5]:
df_1.shape, df_2.shape, df_3.shape, df_4.shape, df_8.shape

((16646, 2), (3340, 2), (5072, 2), (36532, 2), (8854, 2))

In [6]:
df_1.rename(columns = {'label':'class'}, inplace = True)
df_2.rename(columns = {'label':'class'}, inplace = True)
df_3.rename(columns = {'label':'class'}, inplace = True)
df_4.rename(columns = {'label':'class'}, inplace = True)
df_5.rename(columns = {'label':'class'}, inplace = True)
df_6.rename(columns = {'label':'class'}, inplace = True)
df_7.rename(columns = {'label':'class'}, inplace = True)
df_8.rename(columns = {'label':'class'}, inplace = True)

In [7]:
df_fake.shape

(23481, 4)

In [8]:
df_true.shape

(21417, 4)

In [9]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [10]:
df_true.text[0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [11]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


## Inserting a column "class" as target feature

In [12]:
df_fake["class"] = 0
df_true["class"] = 1

In [13]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [14]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = df_fake.tail(10)
df_fake.drop(df_fake.tail(10).index, inplace = True)
    
    
df_true_manual_testing = df_true.tail(10)
df_true.drop(df_true.tail(10).index, inplace = True)

In [15]:
df_fake.shape, df_true.shape

((23471, 5), (21407, 5))

In [16]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

C:\Users\Admin\AppData\Local\Temp\ipykernel_11080\860779283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_11080\860779283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [17]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [18]:
df_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [19]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)

## Merging True and Fake Dataframes

In [20]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [21]:
df_merge.tail(10)

,title,text,subject,date,class
21397,Germany's Schulz says he would demand U.S. wit...,BERLIN (Reuters) - The leader of Germany s Soc...,worldnews,"August 23, 2017",1
21398,Blunt instrument? What a list of banned articl...,SHANGHAI (Reuters) - An old review of an acade...,worldnews,"August 23, 2017",1
21399,Saudi police release teenager detained for dan...,DUBAI (Reuters) - A 14-year-old boy who was de...,worldnews,"August 22, 2017",1
21400,"The People's Princess, Britons work to keep me...",LONDON (Reuters) - Abdul Daoud spilt most of t...,worldnews,"August 23, 2017",1
21401,"Argentina labor unions protest job losses, Mac...",BUENOS AIRES (Reuters) - Argentina s main labo...,worldnews,"August 22, 2017",1
21402,Exclusive: Trump's Afghan decision may increas...,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,worldnews,"August 22, 2017",1
21403,U.S. puts more pressure on Pakistan to help wi...,WASHINGTON (Reuters) - The United States sugge...,worldnews,"August 21, 2017",1
21404,Exclusive: U.S. to withhold up to $290 million...,WASHINGTON (Reuters) - The United States has d...,worldnews,"August 22, 2017",1
21405,Trump talks tough on Pakistan's 'terrorist' ha...,ISLAMABAD (Reuters) - Outlining a new strategy...,worldnews,"August 22, 2017",1
21406,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1


In [22]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

## Removing columns which are not required

In [23]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [24]:
df.isnull().sum()

text     0
class    0
dtype: int64

## Random Shuffling the dataframe

In [25]:
df = pd.concat([df, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8], axis=0)

In [26]:
df

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
8849,sunday screen war democracy st century wire sa...,0
8850,u condemn venezuelan election neither free fai...,1
8851,greek archaeologist clear way hellenikon resor...,1
8852,hometown take obama refugee list city transpla...,0


In [27]:
df = df.sample(frac = 1)

In [28]:
df

,text,class
25299,u lawmaker want restrict internet surveillance...,1
15244,turkey form closer tie iraqi central governmen...,1
9164,propaganda star trek beyond social justice war...,0
15897,macron presidential jet plan hit badpublicity ...,1
26915,israelis throw death arab rally thousand atten...,0
...,...,...
3183,bill reilly back tell fan go away confident tr...,0
34613,hr mcmaster repeatedly refuse say work steve b...,0
1743,small bathroom plane pose challenge passenger ...,1
2835,family roof forgive mercy soul roof lead suspe...,1


In [29]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [30]:
df.columns

Index(['text', 'class'], dtype='object')

In [31]:
df.head()

,text,class
0,u lawmaker want restrict internet surveillance...,1
1,turkey form closer tie iraqi central governmen...,1
2,propaganda star trek beyond social justice war...,0
3,macron presidential jet plan hit badpublicity ...,1
4,israelis throw death arab rally thousand atten...,0


## Creating a function to process the texts

In [32]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) # removes occurences of text such as [abc], [123], [xyz123]
    text = re.sub("\\W"," ",text) # replaces all non-word characters (e.g. punctuation marks, special characters)
    text = re.sub('https?://\S+|www\.\S+', '', text) #  will match both URLs that start with "http://" or "https://" and URLs that start with "www"
    text = re.sub('<.*?>+', '', text) # the regular expression pattern <.*?>+ will match all HTML tags
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # removes punctuations
    text = re.sub('\n', '', text) # removes next line
    text = re.sub('\w*\d\w*', '', text) # removes any string containing digits
    return text

In [33]:
df["text"] = df["text"].apply(wordopt)

In [34]:
df.to_csv("final.csv")

## Defining dependent and independent variables

In [37]:
x = df["text"]
y = df["class"]

## Splitting Training and Testing

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

## Convert text to vectors

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## Logistic Regression

In [40]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

C:\Users\shvmt\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [41]:
pred_lr=LR.predict(xv_test)

In [42]:
LR.score(xv_test, y_test)

0.9619160571918034

In [43]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     15575
           1       0.97      0.95      0.96     14779

    accuracy                           0.96     30354
   macro avg       0.96      0.96      0.96     30354
weighted avg       0.96      0.96      0.96     30354



## Decision Tree Classification

In [44]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [45]:
pred_dt = DT.predict(xv_test)

In [46]:
DT.score(xv_test, y_test)

0.9585886538841668

In [47]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     15575
           1       0.96      0.95      0.96     14779

    accuracy                           0.96     30354
   macro avg       0.96      0.96      0.96     30354
weighted avg       0.96      0.96      0.96     30354



## Gradient Boosting Classifier

In [48]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [49]:
pred_gbc = GBC.predict(xv_test)

In [50]:
GBC.score(xv_test, y_test)

0.9550306384661

In [51]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.94      0.97      0.96     15575
           1       0.97      0.94      0.95     14779

    accuracy                           0.96     30354
   macro avg       0.96      0.95      0.95     30354
weighted avg       0.96      0.96      0.95     30354



## Random Forest Classifier

In [52]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [53]:
pred_rfc = RFC.predict(xv_test)

In [54]:
RFC.score(xv_test, y_test)

0.9521315147921197

In [55]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95     15575
           1       0.96      0.94      0.95     14779

    accuracy                           0.95     30354
   macro avg       0.95      0.95      0.95     30354
weighted avg       0.95      0.95      0.95     30354



In [56]:
import joblib
joblib.dump(LR,'./lr.pkl')
joblib.dump(DT, './dt.pkl')
joblib.dump(GBC, './gbc.pkl')
joblib.dump(RFC, './rfc.pkl')

['./rfc.pkl']

## XGBoost

In [59]:
import xgboost as xgb

XGB = xgb.XGBClassifier(random_state=7)
XGB.fit(xv_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:42:47] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=7,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [60]:
pred_xgb = XGB.predict(xv_test)

In [61]:
XGB.score(xv_test, y_test)

0.9749621137247151

In [62]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95     15575
           1       0.96      0.94      0.95     14779

    accuracy                           0.95     30354
   macro avg       0.95      0.95      0.95     30354
weighted avg       0.95      0.95      0.95     30354



In [63]:
joblib.dump(XGB, './xgb.pkl')

['./xgb.pkl']

## Model Testing

In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    print(new_def_test)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)
    return print(f"\n\nLR Prediction: {output_lable(pred_LR[0])} \nDT Prediction: {output_lable(pred_DT[0])} \nGBC Prediction: {output_lable(pred_GBC[0])} \nRFC Prediction: {output_lable(pred_RFC[0])}")

In [ ]:
news = str(input())
manual_testing(news)

In [ ]:
news = str(input())
manual_testing(news)

In [ ]:
news = str(input())
manual_testing(news)